In [4]:
from datetime import datetime
import pathlib
import json
from tab_dataset import Sdataset
import pandas as pd
import ntv_pandas as npd
from ntv_pandas.pandas_ntv_connector import to_analysis
from time import time
from tab_analysis import AnaDataset
from ntv_pandas.pandas_ntv_connector import SeriesConnec, PdUtil
from json_ntv import Ntv

In [5]:
date= '2024-02-07'
#chemin = 'D:\\philippe\\python ESstandard\\Environmental-Sensing\\python\\Validation\\irve\\Analyse\\'
#chemin = 'C:\\Users\\phili\\github\\Environmental-Sensing\\python\\Validation\\irve\\Analyse\\'
chemin = str(pathlib.Path(npd.__file__).parent.parent.parent/"Environmental-Sensing"/"python"/"Validation"/"irve"/"Analyse")
irve = pd.read_csv(chemin + '/' + 'IRVE_itinerance_valide' + date +'.csv', low_memory=False)
print('nombre de lignes : ', len(irve)) 

nombre de lignes :  62200


In [6]:
del irve['Unnamed: 0']

In [7]:
irve.columns

Index(['index', 'nom_amenageur', 'siren_amenageur', 'contact_amenageur',
       'nom_operateur', 'contact_operateur', 'telephone_operateur',
       'nom_enseigne', 'id_station_itinerance', 'id_station_local',
       'nom_station', 'implantation_station', 'adresse_station',
       'code_insee_commune', 'coordonneesXY', 'nbre_pdc', 'id_pdc_itinerance',
       'id_pdc_local', 'puissance_nominale', 'prise_type_ef', 'prise_type_2',
       'prise_type_combo_ccs', 'prise_type_chademo', 'prise_type_autre',
       'gratuit', 'paiement_acte', 'paiement_cb', 'paiement_autre',
       'tarification', 'condition_acces', 'reservation', 'horaires',
       'accessibilite_pmr', 'restriction_gabarit', 'station_deux_roues',
       'raccordement', 'num_pdl', 'date_mise_en_service', 'observations',
       'date_maj', 'cable_t2_attache', 'last_modified', 'datagouv_dataset_id',
       'datagouv_resource_id', 'datagouv_organization_or_owner', 'created_at',
       'consolidated_longitude', 'consolidated_latitud

In [8]:
t0 = time()
dts = Sdataset(irve) # 1.45
t1 = time()
dic2 = dts.to_analysis() # 18.7
t11 = time()
tr = dts.tree() # 30.7
t2 = time()
tr = dts.tree() # 8.4
print(t1-t0, t11-t1, t2-t11, time()-t2)

1.4514503479003906 18.679101705551147 30.75283646583557 8.402432680130005


In [9]:
t0 = time()
anapd = irve.npd.analysis() # 9.3
t1 = time()
tr = anapd.tree() # 8.0
print(t1-t0, time()-t1)

9.343283891677856 8.033559083938599


In [10]:
t0 = time()
ntv = dts.to_ntv(modecodec='full') # 127.7
t1 = time()
js = ntv.to_obj(encoded=True) # 24.5
print(t1-t0, time()-t1)
len(js)

127.70713019371033 24.543725728988647


58991081

In [11]:
ntv = dts.to_ntv(modecodec='default')
js = ntv.to_obj(encoded=True)
len(js)

18140435

In [21]:
dts.setcanonorder()

Sdataset[62200, 58]

In [12]:
ntv = dts.to_ntv()
js = ntv.to_obj(encoded=True)
len(js)

14624423

In [13]:
js = ntv.to_obj(encoded=True, format='cbor')
len(js)

8266175

In [14]:
t0 = time()
pdjs = irve.to_json() # 0.49
print(time()-t0)
len(pdjs)

0.48236894607543945


79072757

In [15]:
pdjs = irve.to_json(orient='values')
len(pdjs)

50979536

In [16]:
t0 = time()
srs = irve[irve.columns[1]]
js = SeriesConnec.to_json_ntv(PdUtil.unic(srs))[0]
t1 = time()
print(t1-t0) # 1.39
ntv_type = 'json'
ntv_val = PdUtil.ntv_val(ntv_type, srs)
t2 = time()
ntv_value = Ntv.from_obj(ntv_val, def_type=ntv_type).ntv_value
t3 = time()
print(t2-t1, t3-t2)  # 0.0 0.63


1.3890106678009033
0.0004982948303222656 0.6277337074279785


In [17]:
t0 = time()
pdjs = irve.npd.to_json() # 157.9
print(time()-t0)
len(pdjs)

157.92333483695984


1

In [18]:
t0 = time()
value = irve
df2 = value.reset_index() # 0.06
t1 = time()
lis = [SeriesConnec.to_json_ntv(PdUtil.unic(df2[col]))[0] for col in df2.columns] # 88.7
t2 = time()
ntv = Ntv.obj(lis) # 50.26
t3 = time()
jsn = ntv.to_obj() # 21.9
t4 = time()
txt = json.dumps(jsn) # 0.58
t5 = time()
print(t1-t0, t2-t1, t3-t2, t4-t3, t5-t4)

0.06272411346435547 88.6942343711853 50.25679683685303 21.90287733078003 0.5794548988342285


In [19]:
t0 = time()
pdjs = irve.npd.to_json(encoded=True)
print(time()-t0) # 166.3
len(pdjs) # 51.86

166.30137634277344


51860188

In [20]:
irve_cat = irve.astype("category")
t0 = time()
pdjs = irve_cat.npd.to_json(encoded=True) # 224.31
print(time()-t0)
len(pdjs) # 19.45

224.31744623184204


19447887

### synthèse
- csv : 43.5


- pandas (default): 80.6 - 79.1
- pandas (values) : 51.8 - 51


- npd full : 52.7 - 51.86
- npd cat : 20.2 - 19.45


- Ntv full : 59.4 - 59
- Ntv default : 18.9 - 18.1
- Ntv optimized : 14.9 - 14.6
- cbor optimized : 8.6 - 8.3


- parquet : 2.4